In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
inpt_path = 'Cornell Data.xlsx'

In [3]:
Tbill = pd.read_excel(io=inpt_path, sheet_name='T-Bill')
TbillETF = pd.read_excel(io=inpt_path, sheet_name='1-3 MONTH ETF')

# convert yield to price
Tbill['Mid Price'] = Tbill['Mid Price'].apply(lambda x:100/(1+x/12))

In [4]:
AggIndex = pd.read_excel(io=inpt_path, sheet_name='Bloomberg Agg Index')
AggETF = pd.read_excel(io=inpt_path, sheet_name='Barclays Agg ETF')

In [5]:
SP500 = pd.read_excel(io=inpt_path, sheet_name='S&P500')

In [6]:
Russell1000 = pd.read_excel(io=inpt_path, sheet_name='Russell 1000')
Russell1000ETF = pd.read_excel(io=inpt_path, sheet_name='russell 1000 etf')

In [7]:
EmergingETF = pd.read_excel(io=inpt_path, sheet_name='SPDR ETF')

### 1. Portfolio using Index as Instruments

In [8]:
Portfolio_Classic = Tbill.merge(AggIndex, on='Date')
Portfolio_Classic = Portfolio_Classic.merge(SP500, on='Date')
Portfolio_Classic = Portfolio_Classic.merge(Russell1000, on='Date')

In [9]:
Portfolio_Classic.columns = ['Date', 'T-Bill', 'US AggIndex', 'S&P 500', 'Russell 1000']

In [10]:
Portfolio_Classic = Portfolio_Classic.set_index('Date')
Portfolio_Classic = Portfolio_Classic.sort_index(ascending=True)

In [41]:
Portfolio_Classic

,T-Bill,US AggIndex,S&P 500,Russell 1000
Date,,,,
2010-01-25,99.833611,1561.25,1096.79,602.824
2010-01-26,99.833611,1560.98,1092.17,600.345
2010-01-27,99.916736,1560.28,1097.50,602.952
2010-01-28,99.916736,1560.05,1084.53,595.636
2010-01-29,99.833611,1563.87,1073.87,589.410
2010-02-01,99.585062,1561.40,1089.19,598.113
2010-02-02,99.667774,1563.18,1103.32,605.956
2010-02-03,99.585062,1560.28,1097.28,602.542
2010-02-04,99.667774,1565.27,1063.11,583.553


In [54]:
# calculate the daily return for each Index instrument 
PotfReturn_Classic = Portfolio_Classic.apply(lambda x:(x/x.shift()-1))

In [55]:
PotfReturn_Classic.index

DatetimeIndex(['2010-01-25', '2010-01-26', '2010-01-27', '2010-01-28',
               '2010-01-29', '2010-02-01', '2010-02-02', '2010-02-03',
               '2010-02-04', '2010-02-05',
               ...
               '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14',
               '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-21',
               '2020-01-22', '2020-01-23'],
              dtype='datetime64[ns]', name='Date', length=2498, freq=None)

In [56]:
PotfReturn_Classic = PotfReturn_Classic.dropna()

In [57]:
PotfReturn_Classic

,T-Bill,US AggIndex,S&P 500,Russell 1000
Date,,,,
2010-01-26,0.000000,-0.000173,-0.004212,-0.004112
2010-01-27,0.000833,-0.000448,0.004880,0.004343
2010-01-28,0.000000,-0.000147,-0.011818,-0.012134
2010-01-29,-0.000832,0.002449,-0.009829,-0.010453
2010-02-01,-0.002490,-0.001579,0.014266,0.014766
2010-02-02,0.000831,0.001140,0.012973,0.013113
2010-02-03,-0.000830,-0.001855,-0.005474,-0.005634
2010-02-04,0.000831,0.003198,-0.031141,-0.031515
2010-02-05,0.000831,0.002064,0.002897,0.002706


In [69]:
def portfolioReturns(initi, equityp, bondp):
    value = initi
    lv = [initi]
    years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
    r = Portfolio_Classic.iloc[0, 3] # Russell 1000
    agg = Portfolio_Classic.iloc[0, 1] # US AggIndex
    t = Portfolio_Classic.iloc[0, 0] # T-Bill
    perc_r = equityp
    perc_agg = bondp*0.8
    perc_t = bondp*0.2
    value_r = initi*perc_r 
    value_agg = initi*perc_agg
    value_t = initi*perc_t
    shares_r = value_r/r
    shares_agg = value_agg/agg
    shares_t = value_t/t
    for year in years: 
        for index, row in PotfReturn_Classic[PotfReturn_Classic.index.year == year].iterrows():
            r = Portfolio_Classic.loc[index, 'Russell 1000']
            agg = Portfolio_Classic.loc[index, 'US AggIndex']
            t = Portfolio_Classic.loc[index, 'T-Bill']
            value_r = shares_r*r
            value_agg = shares_agg*agg
            value_t = shares_t*t
            value = value_r + value_agg + value_t
            lv.append(value)
        value_r = value*perc_r
        value_agg = value*perc_agg
        value_t = value*perc_t
        shares_r = value_r/r
        shares_agg = value_agg/agg
        shares_t = value_t/t
    for index, row in PotfReturn_Classic[PotfReturn_Classic.index.year == 2020].iterrows():
            r = Portfolio_Classic.loc[index, 'Russell 1000']
            agg = Portfolio_Classic.loc[index, 'US AggIndex']
            t = Portfolio_Classic.loc[index, 'T-Bill']
            value_r = shares_r*r
            value_agg = shares_agg*agg
            value_t = shares_t*t
            value = value_r + value_agg + value_t
            lv.append(value)
    return lv    
        
        
            
            
    
    
    

In [71]:
l = portfolioReturns(1000, 0.9, 0.1)

In [72]:
Portfolio_Classic['$1000 initial investment - 90/10 split'] = l

In [73]:
Portfolio_Classic

,T-Bill,US AggIndex,S&P 500,Russell 1000,$1000 initial investment - 90/10 split
Date,,,,,
2010-01-25,99.833611,1561.25,1096.79,602.824,1000.000000
2010-01-26,99.833611,1560.98,1092.17,600.345,996.285085
2010-01-27,99.916736,1560.28,1097.50,602.952,1000.158050
2010-01-28,99.916736,1560.05,1084.53,595.636,989.223673
2010-01-29,99.833611,1563.87,1073.87,589.410,980.107511
2010-02-01,99.585062,1561.40,1089.19,598.113,992.924497
2010-02-02,99.667774,1563.18,1103.32,605.956,1004.741664
2010-02-03,99.585062,1560.28,1097.28,602.542,999.479485
2010-02-04,99.667774,1565.27,1063.11,583.553,971.401682


In [13]:
# calculate the rolling std of daily return for each Index instrument 
PotfStd_Classic = PotfReturn_Classic.rolling(250).std()

In [14]:
PotfStd_Classic

,T-Bill,US AggIndex,S&P 500,Russell 1000
Date,,,,
2010-01-25,NaN,NaN,NaN,NaN
2010-01-26,NaN,NaN,NaN,NaN
2010-01-27,NaN,NaN,NaN,NaN
2010-01-28,NaN,NaN,NaN,NaN
2010-01-29,NaN,NaN,NaN,NaN
2010-02-01,NaN,NaN,NaN,NaN
2010-02-02,NaN,NaN,NaN,NaN
2010-02-03,NaN,NaN,NaN,NaN
2010-02-04,NaN,NaN,NaN,NaN


### 2. Portfolio using ETFs as Instruments

In [15]:
Portfolio_ETF = TbillETF.merge(AggETF, on='Date', how='outer')
Portfolio_ETF = Portfolio_ETF.merge(EmergingETF, on='Date')
Portfolio_ETF = Portfolio_ETF.merge(Russell1000ETF, on='Date')

In [16]:
Portfolio_ETF.columns = ['Date', 'T-Bill ETF', 'US AggETF', 'Emerging Market ETF', 'Russell 1000 ETF']

In [17]:
Portfolio_ETF = Portfolio_ETF.set_index('Date')
Portfolio_ETF = Portfolio_ETF.sort_index(ascending=True)

In [18]:
# calculate the daily return for each ETF instrument 
PotfReturn_ETF = Portfolio_ETF.apply(lambda x:(x/x.shift()-1))

In [19]:
# calculate the rolling std of daily return for each ETF instrument 
PotfStd_ETF = PotfReturn_ETF.rolling(250).std()

In [20]:
PotfStd_ETF

,T-Bill ETF,US AggETF,Emerging Market ETF,Russell 1000 ETF
Date,,,,
2009-12-31,NaN,NaN,NaN,NaN
2010-01-04,NaN,NaN,NaN,NaN
2010-01-05,NaN,NaN,NaN,NaN
2010-01-06,NaN,NaN,NaN,NaN
2010-01-07,NaN,NaN,NaN,NaN
2010-01-08,NaN,NaN,NaN,NaN
2010-01-11,NaN,NaN,NaN,NaN
2010-01-12,NaN,NaN,NaN,NaN
2010-01-13,NaN,NaN,NaN,NaN
